data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data

Folder structure:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 0-999 in data/train/dogs
- put the dog pictures index 1000-1400 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...

In [ ]:
#import all required libraries
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

In [ ]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = '/content/drive/MyDrive/data/train'
validation_data_dir = '/content/drive/MyDrive/data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16


## Using the bottleneck features of a pre-trained network

In [ ]:
#using here Transfer Learning
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet') #load the Pretrained VGG-16 Model with Imagenet weights (Without top layer)

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save('bottleneck_features_train.npy',bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save('bottleneck_features_validation.npy',bottleneck_features_validation)

# run this model on our training and validation data once, recording the output (the "bottleneck features" from the VGG16 model)

def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load('bottleneck_features_validation.npy')
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

#will train a small fully-connected model on top of the stored features
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path) 


save_bottlebeck_features()
train_top_model()

Found 2000 images belonging to 2 classes.


<ipython-input-7-e675c8346354>:13: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_train = model.predict_generator(


Found 802 images belonging to 2 classes.


<ipython-input-7-e675c8346354>:23: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_validation = model.predict_generator(


Epoch 1/50
125/125 [==============================] - 4s 25ms/step - loss: 0.6311 - accuracy: 0.7610 - val_loss: 0.4334 - val_accuracy: 0.7738
Epoch 2/50
125/125 [==============================] - 4s 31ms/step - loss: 0.3309 - accuracy: 0.8615 - val_loss: 0.3160 - val_accuracy: 0.8625
Epoch 3/50
125/125 [==============================] - 3s 28ms/step - loss: 0.2794 - accuracy: 0.8895 - val_loss: 0.3226 - val_accuracy: 0.8700
Epoch 4/50
125/125 [==============================] - 3s 24ms/step - loss: 0.2258 - accuracy: 0.9105 - val_loss: 0.2852 - val_accuracy: 0.8963
Epoch 5/50
125/125 [==============================] - 3s 24ms/step - loss: 0.2259 - accuracy: 0.9135 - val_loss: 0.2510 - val_accuracy: 0.9013
Epoch 6/50
125/125 [==============================] - 3s 26ms/step - loss: 0.1883 - accuracy: 0.9295 - val_loss: 0.2666 - val_accuracy: 0.9025
Epoch 7/50
125/125 [==============================] - 4s 32ms/step - loss: 0.1652 - accuracy: 0.9345 - val_loss: 0.3287 - val_accuracy: 0.9013

## Conclusion
1. Reached validation accuracy of 0.88 after 50 epoch,not bad at all.
 